# Running simulations

The basic functionality for performing the AWS simulations is implemented by the `aws` package. With this in place, running simulations is a matter of setting up the simulations with the desired parameters and then executing them via IPyParallel.

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipyparallel as ipp
import os
from aws.retrieval import Simulation, Retrieval
from aws.data import RandomProfile
from aws.sensor import ATMS, AWS
from aws import aws_path

## Connecting to IPyParallel

Assuming that the IPyParallel controller, hub and engines are running we create a client, which holds the connection to the controller.

In [8]:
from aws.retrieval import Retrieval
client = ipp.Client(profile="gold")

Since we are already running 4 engines on each of the gold machines, which only have 4 cores, we set the `OMP_NUM_THREADS` environment variable to 1 on all engines to avoid ARTS from launching to many threads.

Note that this notebook is running on your local machine, not on the gold cluster. To execute a cell on the engines we need to use
the  `%%px` cell magic, which executes the cell on the remote engines instead on the local machine.

In [9]:
%%px
%env OMP_NUM_THREADS=1

[stdout:0] env: OMP_NUM_THREADS=1
[stdout:1] env: OMP_NUM_THREADS=1
[stdout:2] env: OMP_NUM_THREADS=1
[stdout:3] env: OMP_NUM_THREADS=1
[stdout:4] env: OMP_NUM_THREADS=1
[stdout:5] env: OMP_NUM_THREADS=1
[stdout:6] env: OMP_NUM_THREADS=1
[stdout:7] env: OMP_NUM_THREADS=1


## Setting up the simulation

The next step is to setup the simulation with the desired configuration.

## Choosing the sensor

Currently two sensor configurations are available:

  * `ATMS`: Simplified version of the upper channels of the
     ATMS sensor.
  * `AWS`: The channel setup to use for the full simulations.

In [10]:
sensor = ATMS()

## Setting the ice habit

The ice shape to use in the simulations. Shoule be one of
* `"Perpendicular3BulletRosette"`
* `"LargePlateAggregate"`
* `"LargeColumnAggregate"`

In [11]:
ice_shape = "Perpendicular3BulletRosette"

## Setting up the simulation



In [12]:
# The input data.
data_provider = RandomProfile("~/Dendrite/Projects/AWS-325GHz/CasesV1")

# Setup the IWC/RWC retrieval, which then becomes the
# data_provider for the simulation.
retrieval = Retrieval(data_provider, ice_shape)

# Setup the simulation
simulation = Simulation(sensor, retrieval, ice_shape)
inputs = [("filename", ("name_length",)),
          ("profile_index", ())]

## Output file

In [13]:
filename = os.path.join(aws_path, "data", "simulations_allsky_v1.nc")
simulation.initialize_output_file(filename,
                                 [("cases", -1, 0)], # Name of dimension for different simulations
                                 inputs=inputs)

## Running the simulations

Running the simulations is performed by executing the `run_ranges` method which distributes the simulations for the given range of profiles on the IPyParallel cluster.

In [15]:
n = 24 # Number of simulations to run
results = simulation.run_ranges(range(n), ipyparallel_client=client)

Simulations are executed asynchronously, which means the `run_ranges` will return berfore the simulations are finished. To monitor progress you can print the results object, which will display the amount of completed simulations. If any simulations failed, the exceptions can be accessed via the `results.failed` attribute.

> **NOTE:** The profile order is random but fixed. Running `simulation.run_ranges(1000)` multiple times will produce the same
 results. 

In [20]:
results

Distributed ARTS simulation: 24 tasks, 24 completed, 0 failed
	 Avg. execution time: 0:00:46.876864

## Running clearsky simulations

Running clearsky simulations is performed in a separate simulation. All that is required to run clearsky versions of the simulations is to pass the `clearsky=True` keyword argument to the `run_ranges` method.

Note, however, that you will need to write the results to a different file, otherwise the results from the allsky simulations will be overwritten

In [21]:
filename = os.path.join(aws_path, "data", "simulations_clearsky_v1.nc")
simulation.initialize_output_file(filename,
                                 [("cases", -1, 0)], # Name of dimension for different simulations
                                 inputs=inputs)
results = simulation.run_ranges(range(n),
                                ipyparallel_client=client,
                               clearsky=True)

In [ ]:
results